# Add defects to a material

Add defects

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials


## Methodology

The following happens in the script below:




## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials

In [10]:

DEFECTS_PARAMETERS = {
    "TYPE": "interstitial",  # "substitution", "vacancy"
    "SPECIES": "Ge",
    "CONCENTRATION": 0.1,
    
}


## 2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [11]:
import sys
if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb","../../config.yml")

## 3. Load input Materials


In [12]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
# visualize(materials, number_of_repetitions=1)
si = list(filter(lambda m: m.name == "Silicon FCC", materials))[0]

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 2-WS2 has been read successfully.
Data from 3-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from 5-HfO2 has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.105% has been read successfully.
Data from Ni3 C2 has been read successfully.
Data from Ni8 C2 has been read successfully.
Data from Si has been read successfully.


## 4. Create interfaces

### 4.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.

In [13]:
import numpy as np
from mat3ra.made.tools.convert import to_pymatgen, from_pymatgen, to_ase
from mat3ra.made.tools.modify import wrap_to_unit_cell

from pymatgen.analysis.defects.core import Substitution, Vacancy, Interstitial
from pymatgen.core import PeriodicSite, Species, Structure
silicon = to_pymatgen(si).make_supercell([1, 1, 1])

si0_site = silicon[0]
sub_site = PeriodicSite(species=Species("Ge"), coords=si0_site.frac_coords, lattice=silicon.lattice)
# instantiate the defect object
# substitution_defect = Substitution(structure=silicon, site=sub_site)
# vacancy_defect = Vacancy(structure=silicon, site=sub_site)
substitution_defect = Interstitial(structure=silicon, site=sub_site)
print(substitution_defect)

# visualize(material)

def translate_structure(structure, fractional_coordinates=[0, 0, 0]):
    min_c = min(site.c for site in structure)
    translation_vector = fractional_coordinates  # 0.1 to ensure there's a small gap above z=0
    translated_structure = structure.copy()
    for site in translated_structure:
        site.coords += np.dot(translation_vector, structure.lattice.matrix)
    return wrap_to_unit_cell(translated_structure)

structure = translate_structure(substitution_defect.defect_structure, [0.5, 0.5, 0.5])
material = from_pymatgen(structure)
from ase.visualize import view
view(to_ase(material))

Ge intersitial site at [0.00,0.00,0.00]


<Popen: returncode: None args: ['/Users/mat3ra/code/GREEN/api-examples/.venv...>

In [14]:
from pymatgen.analysis.defects.generators import (
    generate_all_native_defects, VoronoiInterstitialGenerator
)
generator = VoronoiInterstitialGenerator(        
        clustering_tol = 0.1,
        min_dist = 0.1,
        ltol = 0.1,
        stol = 0.1,
        angle_tol = 1
)

defects = generator.get_defects(silicon, insert_species=["Ge"])
view(to_ase(from_pymatgen(defects[1].defect_structure))*[2,2,2])
# for defect in generate_all_native_defects(silicon, int_generator=generator):
#     print(defect)

<Popen: returncode: None args: ['/Users/mat3ra/code/GREEN/api-examples/.venv...>

In [15]:
from pymatgen.core.surface import SlabGenerator
import json
from pymatgen.analysis.defects.generators import (
    generate_all_native_defects, VoronoiInterstitialGenerator
)
generator = VoronoiInterstitialGenerator(        
        clustering_tol = 0.1,
        min_dist = 0.1,
        ltol = 0.1,
        stol = 0.1,
        angle_tol = 1
)
slab_json = """
{"name":"Silicon FCC - slab [1,0,0]","basis":{"elements":[{"id":0,"value":"Si"},{"id":1,"value":"Si"},{"id":2,"value":"Si"},{"id":3,"value":"Si"},{"id":4,"value":"Si"},{"id":5,"value":"Si"}],"coordinates":[{"id":0,"value":[0,0,0]},{"id":1,"value":[0,0,0.06666667]},{"id":2,"value":[0,0,0.1333333]},{"id":3,"value":[0.2500001,0.25,0.01666666]},{"id":4,"value":[0.2500001,0.25,0.08333333]},{"id":5,"value":[0.2500001,0.25,0.15]}],"units":"crystal","cell":[[3.34892,0,1.9335],[1.116307,3.157392,1.9335],[0,0,58.005]],"constraints":[]},"lattice":{"a":3.867,"b":3.867,"c":58.005,"alpha":60,"beta":60,"gamma":60,"units":{"length":"angstrom","angle":"degree"},"type":"TRI","vectors":{"a":[3.34892,0,1.9335],"b":[1.116307,3.157392,1.9335],"c":[0,0,58.005],"alat":1,"units":"angstrom"}},"isNonPeriodic":false,"metadata":{"isSlab":true,"h":1,"k":0,"l":0,"thickness":3,"vacuumRatio":0.8,"vx":1,"vy":1},"_id":"","isUpdated":true}
"""
material_slab = Material(json.loads(slab_json))
silicon_slab = to_pymatgen(material_slab)
defects = generator.get_defects(silicon_slab, insert_species=["Ge"])


generator = VoronoiInterstitialGenerator(        
        clustering_tol = 0.1,
        min_dist = 0.1,
        ltol = 0.1,
        stol = 0.1,
        angle_tol = 1
)
defects = generator.get_defects(silicon_slab, insert_species=["Ge"])
# visualize(material_slab, rotation="-90x")
# view(to_ase(from_pymatgen(silicon_slab)))
print(defects)
view(to_ase(from_pymatgen(defects[1].defect_structure))*[1,1,1])
visualize(from_pymatgen(defects[1].defect_structure), rotation="-90x")
# for defect in generate_all_native_defects(silicon, int_generator=generator):
#     print(defect)

[Ge intersitial site at [0.13,0.62,0.11], Ge intersitial site at [0.50,0.50,0.10], Ge intersitial site at [0.62,0.63,0.14], Ge intersitial site at [0.75,0.75,0.12], Ge intersitial site at [0.63,0.62,0.08], Ge intersitial site at [0.63,0.63,0.21]]


GridBox(children=(VBox(children=(Label(value='GeSi6 - Material 0', layout=Layout(height='30px', overflow='hidd…

In [16]:
from pymatgen.core.surface import SlabGenerator
import json
from pymatgen.analysis.defects.generators import (
    generate_all_native_defects, VoronoiInterstitialGenerator
)
generator = VoronoiInterstitialGenerator(        
        clustering_tol = 0.1,
        min_dist = 0.1,
        ltol = 0.1,
        stol = 0.1,
        angle_tol = 1
)
slab_json = """
{"name":"Silicon FCC - slab [1,0,0]","basis":{"elements":[{"id":4,"value":"Si"},{"id":5,"value":"Si"}],"coordinates":[{"id":4,"value":[0.2500001,0.25,0.08333333]},{"id":5,"value":[0.2500001,0.25,0.15]}],"units":"crystal","cell":[[3.34892,0,1.9335],[1.116307,3.157392,1.9335],[0,0,58.005]],"constraints":[]},"lattice":{"a":3.867,"b":3.867,"c":58.005,"alpha":60,"beta":60,"gamma":60,"units":{"length":"angstrom","angle":"degree"},"type":"TRI","vectors":{"a":[3.34892,0,1.9335],"b":[1.116307,3.157392,1.9335],"c":[0,0,58.005],"alat":1,"units":"angstrom"}},"isNonPeriodic":false,"metadata":{"isSlab":true,"h":1,"k":0,"l":0,"thickness":3,"vacuumRatio":0.8,"vx":1,"vy":1},"_id":"","isUpdated":true}
"""
material_slab = Material(json.loads(slab_json))
silicon_slab = to_pymatgen(material_slab)
defects = generator.get_defects(silicon_slab, insert_species=["Ge"])


generator = VoronoiInterstitialGenerator(        
        clustering_tol = 0.1,
        min_dist = 0.1,
        ltol = 0.1,
        stol = 0.1,
        angle_tol = 1
)
defects = generator.get_defects(silicon_slab, insert_species=["Ge"])
# visualize(material_slab, rotation="-90x")
# view(to_ase(from_pymatgen(silicon_slab)))
print(defects)
view(to_ase(from_pymatgen(defects[1].defect_structure))*[1,1,1])
visualize(([from_pymatgen(d.defect_structure) for d in defects]), rotation="-90x")
# for defect in generate_all_native_defects(silicon, int_generator=generator):
#     print(defect)

[Ge intersitial site at [0.75,0.75,0.12], Ge intersitial site at [0.00,0.00,0.13], Ge intersitial site at [0.96,0.96,0.61]]


GridBox(children=(VBox(children=(Label(value='GeSi2 - Material 0', layout=Layout(height='30px', overflow='hidd…

In [17]:
def add_defects(material, settings):
    from pymatgen.analysis.defects.generators import VoronoiInterstitialGenerator
    
    generator = VoronoiInterstitialGenerator(        
            clustering_tol = 0.1,
            min_dist = 0.1,
            ltol = 0.1,
            stol = 0.1,
            angle_tol = 1
    )
    defects = generator.get_defects(to_pymatgen(material), insert_species=[settings["SPECIES"]])
    return [from_pymatgen(d.defect_structure) for d in defects]

defects = add_defects(si, DEFECTS_PARAMETERS)
visualize(defects, rotation="-90x")

GridBox(children=(VBox(children=(Label(value='GeSi2 - Material 0', layout=Layout(height='30px', overflow='hidd…

### 6.3. Pass data to the outside runtime


In [18]:
from utils.jupyterlite import set_data
set_data("materials", materials)

TypeError: 'Material' object is not subscriptable